In [4]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

In [2]:
 

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
_,rvc = load_preprocess(config=config.get('preprocess'))
 

{'preprocess': {'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}, 'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}}}
{'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}


2023-08-13 23:16:04 | INFO | root | Loaded checkpoint '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/vits/model/G_953000.pth' (iteration 630)
2023-08-13 23:16:04 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers
2023-08-13 23:16:04 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-13 23:16:04 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_typ

Models loaded
{'name': 'rvc_speakers', 'hubert_model_path': 'rvc/model/hubert_base.pt', 'rvc_config_file': '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/rvc.yaml'}
/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
Loading model: yiqing
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/arianagrande/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/syz/config.jso

In [6]:
import librosa


async def vc_func(
        rvc, audio_file_path, model_index, pitch_adjust, f0_method, 
        feat_ratio, filter_radius, rms_mix_rate, resample_option 
):
 
    if model_index is None:
        return (None, 'Please select a model.')
      
    audio, sr = librosa.load(audio_file_path, sr=None)  # sr=None 表示保持原始采样率

    # 将 NumPy 数组转换为 Python 列表
    audio_samples_list = audio.tolist()
    data = RvcProcessorData( 
        sample_rate=sr, 
        audio_samples= audio_samples_list ,
        model_index= model_index,
        f0_up_key= pitch_adjust,
        f0_method= f0_method,
        index_rate= feat_ratio,
        filter_radius= filter_radius,
        rms_mix_rate= rms_mix_rate,
        resample_sr= resample_option 
    )
    
    return rvc(data)


In [7]:

loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc.loaded_models
]
loaded_models

['Bilibili - 一清清清',
 'ALL - Bob Sponge',
 'ALL - Ariana Grande',
 'ALL - Stefanie Sun',
 '露露 - 露露',
 '毛毛 - 毛毛']

In [8]:

model_index = loaded_models.index('毛毛 - 毛毛')
model_index

5

In [9]:
 

#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-1
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = 0.9
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=1
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0
 
 

In [10]:
audio_file_path = '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/test.wav'

out_sr, output_audio = await vc_func(rvc,audio_file_path,model_index,f0_up_key,f0_method[3],
                                index_rate, filter_radius, rms_mix_rate, resample_rate)


-1
loading rmvpe model
npy: 0.02606344223022461s, f0: 1.2447278499603271s, infer: 0.46862316131591797s


In [11]:
from scipy.io.wavfile import write as write_wav

# save audio to disk
write_wav(f"test.wav", out_sr, output_audio) 

# play text in notebook
Audio(output_audio, rate=out_sr)